In [ ]:
import numpy as np
import struct 
import os
import plotly.graph_objects as go
import time
from scipy.spatial.transform import Rotation as R

import lgchimera.general as general
from lgchimera.io import read_lidar_bin, read_gt

from lgchimera.kitti_util import process_kitti_gt, load_icp_results

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
kitti_seq = '0034'
start_idx = 0

gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'results', 'p2pl_icp')
lidar_Rs, lidar_ts, positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

Evaluate lidar odometry covariance using ground-truth

In [ ]:
N_POSES = len(gt_enu)

# Ground-truth odometry
# odom[i] transforms i to i+1
gt_odom = [None] * (N_POSES - 1)
for i in range(N_POSES - 1):
    R = gt_Rs[i+1] @ gt_Rs[i].T
    t = gt_enu[i+1] - R @ gt_enu[i]
    gt_odom[i] = (R, t)

In [ ]:
from lgchimera.geom_util import se3_logmap

T = np.eye(4)
T[:3, :3] = lidar_Rs[0]
T[:3, 3] = lidar_ts[0]
se3_logmap(T)

In [ ]:
# Rotation sigma
for i in range(N_POSES - 1):
    print(R)

Compare attitudes to velocities

In [ ]:
def trajectory_plot_trace(Rs, ts, color="red", scale=1.0):
    """Generate plotly plot trace for a 3D trajectory of poses

    Parameters
    ----------
    Rs : np.array (3 x 3 x N)
        Sequence of orientations
    ts : np.array (N x 3)
        Sequence of positions
    
    Returns
    -------
    list
        List containing traces for plotting 3D trajectory
    
    """
    points = go.Scatter3d(x=[ts[:,0]], y=[ts[:,1]], z=[ts[:,2]], showlegend=False)#, mode='markers', marker=dict(size=5))
    xs = []; ys = []; zs = []
    for i in range(len(ts)):
        for j in range(3):
            xs += [ts[i,0], ts[i,0] + scale * Rs[0,j,i], None]
            ys += [ts[i,1], ts[i,1] + scale * Rs[1,j,i], None]
            zs += [ts[i,2], ts[i,2] + scale * Rs[2,j,i], None]
    lines = go.Scatter3d(x=xs, y=ys, z=zs, mode="lines", line=dict(color=color), showlegend=False)
    return [points, lines]

In [ ]:
gt_trace = trajectory_plot_trace(np.array(gt_Rs).reshape(3,3,-1), gt_enu, color="blue", scale=10.0)
fig = go.Figure(data=gt_trace)
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

Read lidar points

In [ ]:
binpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'velodyne_points', 'data')
PC_data_all = read_lidar_bin(binpath)
PC_data = PC_data_all[start_idx:]

Ground truth trajectory

In [ ]:
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_data = read_gt(gtpath)
gt_data = gt_data[start_idx:]
lla = gt_data[:,:3]

In [ ]:
init_heading = gt_data[0][5]
r = R.from_euler('XYZ', [0, 0, init_heading])
R_heading = r.as_matrix()

In [ ]:
accels_local = gt_data[:,11:14]
accels_local

In [ ]:
accels_local = gt_data[:,11:14]
accels_global = np.zeros_like(accels_local)
for i in range(len(accels_global)):
    heading = gt_data[i][5]
    r = R.from_euler('XYZ', [0, 0, init_heading])
    R_heading = r.as_matrix()
    accels_global[i] = R_heading @ accels_local[i]
accels_local

In [ ]:
accels_global

In [ ]:
accels_global = gt_data[:,14:17]

In [ ]:
from lgchimera.general import lla_to_ecef, ecef2enu

ref_lla = lla[0]
ecef = lla_to_ecef(*lla[0])
gt_enu = np.zeros((len(lla),3))


for i in range(len(lla)):
    ecef = lla_to_ecef(*lla[i])
    gt_enu[i] = ecef2enu(ecef[0], ecef[1], ecef[2], ref_lla[0], ref_lla[1], ref_lla[2])

gt_enu = gt_enu[:,[1,0,2]]

In [ ]:
import plotly.figure_factory as ff

v_en = gt_data[:,6:8]
N = 200
d = 2

fig = ff.create_quiver(x=gt_enu[0:N:d,0], y=gt_enu[0:N:d,1], u=v_en[0:N:d,1], v=v_en[0:N:d,0], scale=1.0, )
gt_traj = go.Scatter(x=gt_enu[0:N:d,0], y=gt_enu[0:N:d,1], hovertext=np.arange(N), name='Ground-truth')
fig.add_trace(gt_traj)
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
import plotly.figure_factory as ff

N = 100
d = 1

fig = ff.create_quiver(x=gt_enu[0:N:d,0], y=gt_enu[0:N:d,1], u=accels_global[0:N:d,1], v=accels_global[0:N:d,0], scale=1.0, )
gt_traj = go.Scatter(x=gt_enu[0:N:d,0], y=gt_enu[0:N:d,1], hovertext=np.arange(N), name='Ground-truth')
fig.add_trace(gt_traj)
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
fig = ff.create_quiver(x=gt_enu[0:100:1,0], y=gt_enu[0:100:1,1], u=accels_local[0:100:1,0], v=accels_local[0:100:1,1], scale=1.0, )
gt_traj = go.Scatter(x=gt_enu[0:100:1,0], y=gt_enu[0:100:1,1], hovertext=np.arange(N), name='Ground-truth')
fig.add_trace(gt_traj)
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
N = len(gt_enu)
gt_traj = go.Scatter(x=gt_enu[:,0], y=gt_enu[:,1], hovertext=np.arange(N), name='Ground-truth')
fig = go.Figure(data=[gt_traj])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

LiDAR odometry with ICP

In [ ]:
import open3d as o3d
from lgchimera.registration import initialize_source_and_target, p2p_ICP, p2pl_ICP, p2pl_ICP_with_covariance

In [ ]:
from scipy.spatial.transform import Rotation as R

heading = gt_data[0][5] # heading angle
r = R.from_euler('XYZ', [0, 0, heading])
R_heading = r.as_matrix()

In [ ]:
# Load registration results from file
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', '2011_09_30_drive_0028_sync', 'results', 'p2pl_icp')
lidar_Rs = np.load(os.path.join(data_path, 'lidar_Rs_start_1550.npy'))
lidar_ts = np.load(os.path.join(data_path, 'lidar_ts_start_1550.npy'))
positions = np.load(os.path.join(data_path, 'positions_start_1550.npy'))
N = len(lidar_Rs)

In [ ]:
N = len(PC_data)
#N = 1000
R_abs = R_heading
t_abs = gt_ecef[0].copy()
poses = N * [None]
poses[0] = (R_abs.copy(), t_abs.copy())

lidar_Rs = []
lidar_ts = []
lidar_covariances = []

for i in range(1,N):
    print(i, "/", N)
    trans_init = np.eye(4)
    threshold = 1
    source, target = initialize_source_and_target(PC_data[i], PC_data[i-1])
    #reg_p2p, eval_time = p2pl_ICP(source, target, threshold, trans_init)
    reg_p2p, covariance = p2pl_ICP_with_covariance(source, target, threshold, trans_init)
    R_hat = reg_p2p.transformation[:3,:3]
    t_hat = reg_p2p.transformation[:3,3]
    # R_hat = lidar_Rs[i-1]
    # t_hat = lidar_ts[i-1]

    lidar_Rs.append(R_hat)
    lidar_ts.append(t_hat)
    lidar_covariances.append(covariance)

    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    poses[i] = (R_abs.copy(), t_abs.copy())

In [ ]:
positions = np.zeros((N,3))
for i in range(N):
    positions[i] = poses[i][1]

In [ ]:
# TODO: Save registration results to file
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', '2011_09_30_drive_0028_sync', 'results', 'p2pl_icp')
np.save(os.path.join(data_path, 'lidar_Rs_start_1550.npy'), np.array(lidar_Rs))
np.save(os.path.join(data_path, 'lidar_ts_start_1550.npy'), np.array(lidar_ts))
np.save(os.path.join(data_path, 'positions_start_1550.npy'), positions)

In [ ]:
lidar_traj = go.Scatter(x=positions[:,0], y=positions[:,1], hovertext=np.arange(N), name='Lidar odometry')
gt_traj = go.Scatter(x=gt_ecef[:N,0], y=gt_ecef[:N,1], hovertext=np.arange(N), name='Ground-truth')
fig = go.Figure(data=[gt_traj, lidar_traj])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
gt_traj3d = go.Scatter3d(x=gt_ecef[:,0], y=gt_ecef[:,1], z=gt_ecef[:,2], marker=dict(size=2), hovertext=np.arange(N), name='Ground-truth')
lidar_traj3d = go.Scatter3d(x=positions[:,0], y=positions[:,1], z=positions[:,2], marker=dict(size=2), hovertext=np.arange(N), name='Lidar odometry')
fig = go.Figure(data=[gt_traj3d, lidar_traj3d])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

## Factor Graph test

Lidar odometry and simulated GPS with noisy ground-truth

In [ ]:
from lgchimera.pose_graph import PoseGraph

g = PoseGraph()
N_nodes = 1000

# Add initial node
R_abs = R_heading.copy()
t_abs = gt_ecef[0].copy()
g.add_node(1, (R_abs, t_abs))

# GPS noise parameters
gps_sigma = 0.2  # [m]

# ICP parameters
trans_init = np.eye(4)
threshold = 1

# For each new frame
for i in range(1,N_nodes):
    #print(i)

    # Perform LiDAR odometry
    # source, target = initialize_source_and_target(PC_data[i], PC_data[i-1])
    # reg_p2p, eval_time = p2pl_ICP(source, target, threshold, trans_init)
    # R_hat = np.array(reg_p2p.transformation[:3,:3])
    # t_hat = np.array(reg_p2p.transformation[:3,3])
    R_hat = np.array(lidar_Rs[i-1])
    t_hat = np.array(lidar_ts[i-1])

    # Initialize new node with LiDAR odometry estimate
    R_abs = R_hat @ R_abs
    t_abs += R_abs @ t_hat
    g.add_node(i+1, (R_abs, t_abs))

    # Add LiDAR odometry edge
    g.add_edge([i, i+1], (R_hat, t_hat), information=np.eye(6))

    # Simulate GPS measurement
    gps_pos = gt_ecef[i] + np.random.normal(0, gps_sigma, 3)

    # Add GPS factor and edge
    g.add_factor(i+1, (np.eye(3), gps_pos))

In [ ]:
gt_traj3d = go.Scatter3d(x=gt_ecef[:N_nodes,0], y=gt_ecef[:N_nodes,1], z=gt_ecef[:N_nodes,2], marker=dict(size=2), hovertext=np.arange(N), name='Ground-truth')
fig = go.Figure(data=g.plot_trace()+[gt_traj3d])
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
g.optimize()

In [ ]:
fig = go.Figure(data=g.plot_trace()+[gt_traj3d])
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

## Window-based optimization

In [ ]:
from scipy.stats import chi2
import matplotlib.pyplot as plt

In [ ]:
alpha = 0.001
dfs = np.arange(1, 300)
Ts = chi2.ppf(1-alpha, df=dfs)
#plt.plot(dfs, Ts)
fig = go.Figure(data=go.Scatter(x=dfs, y=Ts))
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Chimera epoch - every 3 minutes = 180 seconds = 1800 frames
# Window size: 10 seconds = 100 frames
from lgchimera.pose_graph import PoseGraph

g = PoseGraph()
N_nodes = 100
window_size = 10

# Compute threshold
alpha = 0.001
T = chi2.ppf(1-alpha, df=3*window_size)
print("Threshold = ", T)

# Add initial node
R_abs = R_heading.copy()
t_abs = gt_ecef[0].copy()
g.add_node(1, (R_abs, t_abs))

# GPS noise parameters
gps_sigma = 0.2  # [m]

gps_spoofing_biases = np.arange(0, 10, 0.1)

# ICP parameters
trans_init = np.eye(4)
threshold = 1

# For each new frame
for i in range(1,N_nodes):
    start_time = time.time()

    # Get LiDAR odometry
    R_hat = np.array(lidar_Rs[i-1])
    t_hat = np.array(lidar_ts[i-1])

    # Initialize new node with LiDAR odometry estimate
    R_abs = R_hat @ R_abs
    t_abs += R_abs @ t_hat
    g.add_node(i+1, (R_abs, t_abs))

    # Add LiDAR odometry edge
    lidar_information = np.eye(6)  # TODO: compute information matrix
    g.add_edge([i, i+1], (R_hat, t_hat), information=lidar_information)

    # Simulate GPS measurement
    gps_pos = gt_ecef[i] + np.random.normal(0, gps_sigma, 3) 
    gps_pos[0] += gps_spoofing_biases[i]

    # Add GPS factor and edge
    gps_information = np.eye(6)
    gps_information[:3,:3] *= 1/gps_sigma**2
    gps_information[3:,3:] *= 1e-5  # Scale down information matrix for orientation 
    g.add_factor(i+1, (np.eye(3), gps_pos), information=gps_information)

    # Optimize
    g.optimize(window=(i+1-window_size, i+1))

    # Compute test statistic
    q = g.test_statistic()

    # Time each iteration
    print(i, "/", N_nodes, ": t=", time.time() - start_time, "s, q=", q)
    

In [ ]:
alpha = 0.1
T = chi2.ppf(1-alpha, df=3*window_size)
T

In [ ]:
positions = g.get_positions()

In [ ]:

lidar_traj = go.Scatter(x=positions[:,0], y=positions[:,1], hovertext=np.arange(N), name='Factor graph')
gt_traj = go.Scatter(x=gt_ecef[:N_nodes,0], y=gt_ecef[:N_nodes,1], hovertext=np.arange(N), name='Ground-truth')
fig = go.Figure(data=[gt_traj, lidar_traj])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()